In [1]:
print('working?')

working?


In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import cv2
from multiprocessing import Pool, cpu_count

# ======================== 全局配置 ========================
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# SAM2 初始化
from sam2.build_sam import build_sam2_video_predictor
sam2_checkpoint = "checkpoints/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"
predictor = build_sam2_video_predictor(model_cfg, sam2_checkpoint, device=device)


# ======================== 通用函数 ========================
def show_mask(mask, ax, obj_id=None, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        cmap = plt.get_cmap("tab10")
        cmap_idx = 0 if obj_id is None else obj_id
        color = np.array([*cmap(cmap_idx)[:3], 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

# 多进程可调用函数（必须定义在顶层）
def process_frame(args):
    key, idx, video_segments_todo, video_dir, frame_names, prompt_dir = args
    seg_dir = os.path.join(prompt_dir, f"SAM2_seg_mask_{key}")
    vis_dir = os.path.join(prompt_dir, f"SAM2_seg_plot_{key}")

    frame_path = os.path.join(video_dir, frame_names[idx])
    frame_image = Image.open(frame_path)
    combined_mask = None

    for obj_id, mask in video_segments_todo[idx].items():
        mask = np.squeeze(mask) if mask.ndim > 2 else mask
        scaled_mask = mask * (obj_id * 50)

        Image.fromarray(scaled_mask.astype("uint8")).save(
            os.path.join(seg_dir, f"frame_{idx}_obj_{obj_id}.png"), "PNG")

        combined_mask = np.maximum(combined_mask, scaled_mask) if combined_mask is not None else scaled_mask

    if combined_mask is not None:
        Image.fromarray(combined_mask.astype("uint8")).save(
            os.path.join(seg_dir, f"frame_{idx}_combined.png"), "PNG")

    plt.figure(figsize=(6, 4))
    plt.title(f"Frame {idx} ({key})")
    plt.imshow(frame_image, cmap='gray')
    for obj_id, mask in video_segments_todo[idx].items():
        show_mask(np.squeeze(mask) if mask.ndim > 2 else mask, plt.gca(), obj_id=obj_id)
    plt.savefig(os.path.join(vis_dir, f"frame_{idx}_visualization.png"), format="png", dpi=300)
    plt.close()
    print(f"🖼️ Processed frame {idx} for {key}")

# ======================== 主处理逻辑 ========================
def process_one_pid(pid, data_dir, seed_count):
    print(f"\n🚀 Processing {pid} with {seed_count}-seed")

    img_dir = os.path.join(data_dir, f'img_in_jpg_to_sam2_{seed_count}seed', pid)
    label_dir = os.path.join(data_dir, f'label_in_png_to_sam2_{seed_count}seed', pid)
    prompt_dir = os.path.join(data_dir, f'sam2_results_by_pid_{seed_count}seed', pid)    
    csv_map_path = os.path.join(img_dir, f"{pid}_mapping.csv")
    df_map = pd.read_csv(csv_map_path)
    seed_rows = df_map[df_map['category'] == 'seed']
    prompt_idx = sorted(seed_rows['frame_idx'].tolist())
    
    seg_mask_yeslap_dir = os.path.join(prompt_dir, "SAM2_seg_plot_yeslap")
    if os.path.isdir(seg_mask_yeslap_dir) and len(os.listdir(seg_mask_yeslap_dir)) > 0:
        print(f"✅ Results already exist for {pid}, skipping processing.")
        return
    
    os.makedirs(prompt_dir, exist_ok=True)
    

    frame_names = sorted([p for p in os.listdir(img_dir) if p.lower().endswith('.jpg')],
                         key=lambda p: int(os.path.splitext(p)[0]))

    inference_state = predictor.init_state(video_path=img_dir)
    predictor.reset_state(inference_state)

    # Prompt添加掩码
    class_ranges = {    1: (30, 70),     2: (80, 120),   3: (130, 170),     4: (180, 220)}

    for idx in prompt_idx:
        img_path = os.path.join(img_dir, frame_names[idx])
        seg_path = os.path.join(label_dir, str(idx).zfill(5) + '.png')
        try:
            seg_image = Image.open(seg_path).convert("L")
            img_image = Image.open(img_path).convert("L")
        except Exception as e:
            print(f"❌ Error reading image {idx}: {e}")
            continue

        seg_array = np.array(seg_image)
        img_array = np.array(img_image)

        class_masks = {}
        for class_id, (low, high) in class_ranges.items():
            mask = (seg_array >= low) & (seg_array <= high)
            class_mask = np.zeros_like(seg_array, dtype=np.uint8)
            class_mask[mask] = 1
            class_mask = np.array(Image.fromarray(class_mask).resize(img_array.shape[::-1], Image.NEAREST))
            class_masks[class_id] = class_mask

        for ann_obj_id, class_mask in class_masks.items():
            try:
                _, _, _ = predictor.add_new_mask(
                    inference_state=inference_state,
                    frame_idx=idx,
                    obj_id=ann_obj_id,
                    mask=class_mask
                )
            except Exception as e:
                print(f"   ❌ Error in add_new_mask: {e}")
                
        current_pid = seed_rows.loc[seed_rows['frame_idx'] == idx, 'pid'].values
        current_pid_str = current_pid[0] if len(current_pid) > 0 else 'Unknown'
        print(f"✅ Added prompted frame {idx} ({frame_names[idx]}) with {len(class_masks)} classes from pid {current_pid_str}.")        

    # 推理
    video_segments, video_segments_nolap, iou_score = {}, {}, {}
    for out in predictor.propagate_in_video(
        inference_state, iou_score_return=True, max_frame_num_to_track=len(frame_names)):
        out_frame_idx, out_obj_ids, out_mask_logits = out[:3]
        ious = out[3]
        video_segments[out_frame_idx] = {
            obj_id: (out_mask_logits[i] > 0.0).cpu().numpy() for i, obj_id in enumerate(out_obj_ids)}
        iou_score[out_frame_idx] = {
            obj_id: ious[i] for i, obj_id in enumerate(out_obj_ids)}

        frame_masks = {
            obj_id: (out_mask_logits[i] > 0.0).cpu().numpy() * out_mask_logits[i].cpu().numpy()
            for i, obj_id in enumerate(out_obj_ids)}

        stacked_masks = np.stack([frame_masks[obj_id] > 0 for obj_id in frame_masks], axis=0)
        stacked_logits = np.stack([frame_masks[obj_id] for obj_id in frame_masks], axis=0)
        max_logits_indices = np.argmax(stacked_logits, axis=0)

        video_segments_nolap[out_frame_idx] = {
            obj_id: (max_logits_indices == i).astype(np.uint8) & stacked_masks[i]
            for i, obj_id in enumerate(frame_masks)}

    # 保存 IOU
    iou_csv_path = os.path.join(prompt_dir, pid + "_iou_results.csv")
    results = []
    for f_idx in range(len(frame_names)):
        for o_id, iou in iou_score[f_idx].items():
            max_iou = torch.max(iou).item()
            results.append({"frame_idx": f_idx, "obj_id": o_id, "iou": max_iou})
    results_df = pd.DataFrame(results)
    wide_df = results_df.pivot(index="frame_idx", columns="obj_id", values="iou")
    wide_df.columns = [f"iou_id{col}" for col in wide_df.columns]
    wide_df["iou_sum"] = wide_df.sum(axis=1)
    wide_df.to_csv(iou_csv_path, index=True)
    print(f"📄 Saved IOU results to {iou_csv_path}")

    # 保存 mask 图像
    video_segments_dict = {"yeslap": video_segments, "nolap": video_segments_nolap}
    for key in video_segments_dict:
        os.makedirs(os.path.join(prompt_dir, f"SAM2_seg_mask_{key}"), exist_ok=True)
        os.makedirs(os.path.join(prompt_dir, f"SAM2_seg_plot_{key}"), exist_ok=True)
                
    tasks = []
    for key, video_segments_todo in video_segments_dict.items():
        for idx in range(len(frame_names)):
            tasks.append((key, idx, video_segments_todo, img_dir, frame_names, prompt_dir))

    with Pool(min(2, cpu_count())) as pool:
        pool.map(process_frame, tasks)



In [2]:
import os
import re
import torch

def sort_key(pid):
    m = re.search(r'\d+', pid.split('_')[0])
    return (False, int(m.group()), pid.lower()) if m else (True, 0, pid.lower())

# === Config ===
dataset = 'CT_word'
data_dir = os.path.join('notebooks', 'videos', dataset, 'data_in_jpg')
seed_counts = [1]

# Get sorted PIDs from 1-seed folder
ref_dir = os.path.join(data_dir, 'img_in_jpg_to_sam2_1seed')
pids = sorted([f for f in os.listdir(ref_dir) if os.path.isdir(os.path.join(ref_dir, f))], key=sort_key)

# Process each PID across all seeds
for pid in pids:
    print(f"\n=== 🔁 PID: {pid} ===")
    for seed in seed_counts:
        if os.path.exists(os.path.join(data_dir, f'img_in_jpg_to_sam2_{seed}seed', pid)):
            try:
                process_one_pid(pid, data_dir, seed)
            except Exception as e:
                print(f"❌ {pid} ({seed}-seed): {e}")
    torch.cuda.empty_cache()



=== 🔁 PID: word_0002 ===

🚀 Processing word_0002 with 1-seed


frame loading (JPEG): 100%|████████████████████| 71/71 [00:00<00:00, 119.77it/s]
/home/zhongyi/Desktop/SAM2_SEG/sam2/sam2_video_predictor.py:813: UserWarning: /home/zhongyi/Desktop/SAM2_SEG/sam2/_C.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(


✅ Added prompted frame 0 (00000.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 2 (00002.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 4 (00004.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 6 (00006.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 8 (00008.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 10 (00010.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 12 (00012.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 14 (00014.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 16 (00016.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 18 (00018.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 20 (00020.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 22 (00022.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 24 (00024.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 26 (00026.jpg) with 4 classes from pid tt_s137

propagate in video: 100%|███████████████████████| 71/71 [00:31<00:00,  2.28it/s]

📄 Saved IOU results to notebooks/videos/CT_word/data_in_jpg/sam2_results_by_pid_1seed/word_0002/word_0002_iou_results.csv


...

📄 Saved IOU results to notebooks/videos/CT_word/data_in_jpg/sam2_results_by_pid_1seed/word_0148/word_0148_iou_results.csv
🖼️ Processed frame 0 for yeslap
🖼️ Processed frame 17 for yeslap
🖼️ Processed frame 1 for yeslap
🖼️ Processed frame 18 for yeslap
🖼️ Processed frame 2 for yeslap
🖼️ Processed frame 19 for yeslap
🖼️ Processed frame 3 for yeslap
🖼️ Processed frame 20 for yeslap
🖼️ Processed frame 4 for yeslap
🖼️ Processed frame 21 for yeslap
🖼️ Processed frame 5 for yeslap
🖼️ Processed frame 22 for yeslap
🖼️ Processed frame 6 for yeslap
🖼️ Processed frame 23 for yeslap
🖼️ Processed frame 7 for yeslap
🖼️ Processed frame 24 for yeslap
🖼️ Processed frame 8 for yeslap
🖼️ Processed frame 25 for yeslap
🖼️ Processed frame 9 for yeslap
🖼️ Processed frame 26 for yeslap
🖼️ Processed frame 10 for yeslap
🖼️ Processed frame 27 for yeslap
🖼️ Processed frame 11 for yeslap
🖼️ Processed frame 28 for yeslap
🖼️ Processed frame 12 for yeslap
🖼️ Processed frame 29 for yeslap
🖼️ Processed frame 13 for yesl

frame loading (JPEG): 100%|████████████████████| 67/67 [00:00<00:00, 105.64it/s]
/home/zhongyi/Desktop/SAM2_SEG/sam2/sam2_video_predictor.py:813: UserWarning: /home/zhongyi/Desktop/SAM2_SEG/sam2/_C.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(


✅ Added prompted frame 0 (00000.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 2 (00002.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 4 (00004.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 6 (00006.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 8 (00008.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 10 (00010.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 12 (00012.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 14 (00014.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 16 (00016.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 18 (00018.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 20 (00020.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 22 (00022.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 24 (00024.jpg) with 4 classes from pid tt_s1377.
✅ Added prompted frame 26 (00026.jpg) with 4 classes from pid tt_s137

propagate in video: 100%|███████████████████████| 67/67 [01:23<00:00,  1.25s/it]


📄 Saved IOU results to notebooks/videos/CT_word/data_in_jpg/sam2_results_by_pid_1seed/word_0150/word_0150_iou_results.csv
🖼️ Processed frame 0 for yeslap
🖼️ Processed frame 17 for yeslap
🖼️ Processed frame 1 for yeslap
🖼️ Processed frame 18 for yeslap
🖼️ Processed frame 2 for yeslap
🖼️ Processed frame 19 for yeslap
🖼️ Processed frame 3 for yeslap
🖼️ Processed frame 20 for yeslap
🖼️ Processed frame 21 for yeslap🖼️ Processed frame 4 for yeslap

🖼️ Processed frame 5 for yeslap
🖼️ Processed frame 22 for yeslap
🖼️ Processed frame 6 for yeslap
🖼️ Processed frame 23 for yeslap
🖼️ Processed frame 7 for yeslap
🖼️ Processed frame 24 for yeslap
🖼️ Processed frame 8 for yeslap
🖼️ Processed frame 25 for yeslap
🖼️ Processed frame 9 for yeslap
🖼️ Processed frame 26 for yeslap
🖼️ Processed frame 27 for yeslap
🖼️ Processed frame 10 for yeslap
🖼️ Processed frame 28 for yeslap🖼️ Processed frame 11 for yeslap

🖼️ Processed frame 29 for yeslap
🖼️ Processed frame 12 for yeslap
🖼️ Processed frame 13 for yesl

In [3]:
print('working?')

working?


In [4]:
import os
import psutil

# Get the PID of the current process (Jupyter notebook)
pid = os.getpid()
print(f"Current PID: {pid}")




Current PID: 2513393


In [ ]:
!kill -9 {pid}